## Ingesting PDF

In [1]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"

Note: you may need to restart the kernel to use updated packages.


c:\Users\sumit\AppData\Local\Programs\Python\Python312\python.exe: No module named pip


Note: you may need to restart the kernel to use updated packages.


c:\Users\sumit\AppData\Local\Programs\Python\Python312\python.exe: No module named pip


In [2]:
#from langchain_community.document_loaders import UnstructuredPDFLoader#
#from langchain_community.document_loaders import OnlinePDFLoader

from langchain_community.document_loaders.csv_loader import CSVLoader
import csv
csv.field_size_limit(10**6)

from langchain_community.document_loaders import UnstructuredExcelLoader

In [3]:
#local_path = "WEF_The_Global_Cooperation_Barometer_2024.pdf"
local_path_xlsx = "./pull_requests_summary.xlsx"

# Local PDF file uploads
if local_path_xlsx:
  loader_xlsx = UnstructuredExcelLoader(local_path_xlsx, mode="elements")
  data_xlsx = loader_xlsx.load()
  print("xlsx len : ", len(data_xlsx))
else:
  print("Upload a xlsx file")

xlsx len :  2


In [ ]:
local_path_csv = "./pull_requests_summary.csv"
if local_path_csv:
  loader_csv = CSVLoader(file_path=local_path_csv, encoding="utf-8", csv_args={'delimiter': ','})
  data_csv = loader_csv.load()
  print("CSV len : ", len(data_csv))
else:
  print("Upload a csv file")

In [3]:
# Preview first page
data_csv[0].page_content

'Commit ID: 1463a176bf652172560c3893c84b61c86e1b47f5\nChanges: diff --git a/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java b/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java\nindex dd0b50b37..84bb7cd76 100644\n--- a/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java\n+++ b/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java\n@@ -33,5 +33,5 @@ public class FeatureFlagConstants\n \n     public static final String FF_24_4_FIX_EDIT_APPOINTMENT_DATE = "SUP-522225#2024-08";\n \n-    public static final String FF_99_9_FIX_FACILITY_DAY_LEVEL_SCOPE_FOR_NON_UTC_TIMEZONE = "SUP-504432#2024-08";\n+    public static final String FF_99_9_FIX_FACILITY_DAY_LEVEL_SCOPE_FOR_NON_UTC_TIMEZONE = "SUP-518897#2024-08";\n }\nComments: '

In [4]:
data_xlsx[0].page_content

'pull_requests_summary'

In [5]:
data_xlsx[1].page_content

'\n\n\nCommit ID\nChanges\nComments\n\n\n1463a176bf652172560c3893c84b61c86e1b47f5\ndiff --git a/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java b/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java\\nindex dd0b50b37..84bb7cd76 100644\\n--- a/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java\\n+++ b/src/main/java/com/manh/cp/appointment/appointment/constants/FeatureFlagConstants.java\\n@@ -33,5 +33,5 @@ public class FeatureFlagConstants\\n \\n     public static final String FF_24_4_FIX_EDIT_APPOINTMENT_DATE = "SUP-522225#2024-08";\\n \\n-    public static final String FF_99_9_FIX_FACILITY_DAY_LEVEL_SCOPE_FOR_NON_UTC_TIMEZONE = "SUP-504432#2024-08";\\n+    public static final String FF_99_9_FIX_FACILITY_DAY_LEVEL_SCOPE_FOR_NON_UTC_TIMEZONE = "SUP-518897#2024-08";\\n }\n\n\n\n39c8f648fdb08e8717237ae4ffca5b181cd3fd33\ndiff --git a/gradle.properties b/gradle.properties\\nindex 63e4147

## Vector Embeddings

In [6]:
!ollama pull nomic-embed-text

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [3]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED       
nomic-embed-text:latest	0a109f422b47	274 MB	43 minutes ago	
gemma2:latest          	ff02c3702f32	5.4 GB	10 days ago   	
gemma:2b               	b50d6c999e59	1.7 GB	12 days ago   	
llama2:latest          	78e26419b446	3.8 GB	2 weeks ago   	
mistral:latest         	f974a74358d6	4.1 GB	2 weeks ago   	


In [41]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata

In [6]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=8000, chunk_overlap=100)
#chunks_csv = text_splitter.split_documents(data_csv)
chunks_xlsx = text_splitter.split_documents(data_xlsx)

In [7]:
# Add to vector database
persist_directory_xlsx ='./chroma_db_xlsx'
vector_db_xlsx = Chroma.from_documents(
    documents=filter_complex_metadata(chunks_xlsx), 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    persist_directory=persist_directory_xlsx,
    collection_name="local-rag-xlsx"
)

OllamaEmbeddings: 100%|██████████| 84/84 [03:11<00:00,  2.28s/it]


In [8]:
# Add to vector database
persist_directory_csv ='./chroma_db_csv'
vector_db_csv = Chroma.from_documents(
    documents=data_csv, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    persist_directory=persist_directory_csv,
    collection_name="local-rag-csv"
)

OllamaEmbeddings: 100%|██████████| 20/20 [00:48<00:00,  2.40s/it]


## Retrieval

In [9]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [10]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [11]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [12]:
retriever_xlsx = MultiQueryRetriever.from_llm(
    vector_db_xlsx.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

retriever_csv = MultiQueryRetriever.from_llm(
    vector_db_csv.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [13]:
chain_xlsx = (
    {"context": retriever_xlsx, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain_csv = (
    {"context": retriever_csv, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
chain_xlsx.invoke("Count the total unique commit ids?")

OllamaEmbeddings:   0%|          | 0/1 [00:13<?, ?it/s]


ValueError: Error raised by inference API HTTP code: 500, {"error":"llama runner process has terminated: exit status 0xc0000005"}

In [15]:
#chain_csv.invoke("Count the total unique commit ids?")
chain_csv.invoke("which commit id is the most complex?")

OllamaEmbeddings:   0%|          | 0/1 [00:04<?, ?it/s]


ValueError: Error raised by inference API HTTP code: 500, {"error":"llama runner process has terminated: CUDA error"}

In [30]:
chain.invoke("find the 2 most dissimilar commit Ids?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.11s/it]


' The two most dissimilar commit Ids are 513d606ad and 8da9a52e0. These commits appear to be different due to changes in the Java code related to a FacilityLevelUtilization query and properties in the AppointmentEntities.xml file. Specifically, there are additions, deletions, and modifications in the Java code, while the XML file has some new properties added and others removed.'

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()